In [2]:
import zipfile
import os

# List of zip files
zip_files = [
    'Anomaly-Videos-Part-1.zip',
    'Anomaly-Videos-Part-2.zip',
    'Anomaly-Videos-Part-3.zip',
    'Anomaly-Videos-Part-4.zip'
]

# Output directory to extract files
output_dir = 'anomaly_dataset'

# Make sure output directory exists
os.makedirs(output_dir, exist_ok=True)

# Loop to extract each zip file
for zip_file in zip_files:
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        print(f"Extracting {zip_file}...")
        zip_ref.extractall(output_dir)

print("All files extracted successfully!")


Extracting Anomaly-Videos-Part-1.zip...
Extracting Anomaly-Videos-Part-2.zip...
Extracting Anomaly-Videos-Part-3.zip...
Extracting Anomaly-Videos-Part-4.zip...
All files extracted successfully!


In [3]:
pip install opencv-python av imageio[ffmpeg]


  Using cached imageio-2.37.0-py3-none-any.whl.metadata (5.2 kB)
  Using cached imageio_ffmpeg-0.6.0-py3-none-win_amd64.whl.metadata (1.5 kB)
Using cached imageio-2.37.0-py3-none-any.whl (315 kB)
Using cached imageio_ffmpeg-0.6.0-py3-none-win_amd64.whl (31.2 MB)
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install av

Note: you may need to restart the kernel to use updated packages.


In [15]:
import av
import glob
import os
import time
import tqdm
import datetime
import argparse
import numpy as np
from PIL import Image
from tqdm.autonotebook import tqdm


In [ ]:
# def video_to_frame(path,out_path):
#     vidcap = cv2.VideoCapture(path)
#     success,image = vidcap.read()
#     count = 0
#     while success:
#       cv2.imwrite(os.path.join(out_path,"{}.jpg".format(count)), image)
#       success,image = vidcap.read()
#       count += 1

In [ ]:
# def extract_frames(video_path):
#     frames = []
#     video = av.open(video_path)
#     for frame in video.decode(0):
#         yield frame.to_image()

In [16]:
# Function to extract uniformly sampled frames (handles short videos too)
def extract_uniform_frames(video_path, num_frames=16):
    try:
        container = av.open(video_path)
        stream = container.streams.video[0]
        total_frames = stream.frames  # Total frames in video

        if total_frames == 0:  # Handle empty/corrupt videos
            print(f" No frames found in video: {video_path}")
            return []

        # If total frames are fewer than num_frames, adjust
        if total_frames < num_frames:
            frame_indices = np.linspace(0, total_frames - 1, num=total_frames, dtype=int)
        else:
            frame_indices = np.linspace(0, total_frames - 1, num=num_frames, dtype=int)

        frames = []
        idx_set = set(frame_indices)
        for idx, frame in enumerate(container.decode(video=0)):
            if idx in idx_set:
                frames.append(frame.to_image())
            if idx > frame_indices[-1]:
                break

        # Handle very short videos: duplicate last frame to fill up to num_frames
        while len(frames) < num_frames and frames:
            frames.append(frames[-1])

        return frames

    except Exception as e:
        print(f"❌ Error processing {video_path}: {e}")
        return []

In [17]:
# ✅ Main extraction loop based on dataset structure
path = 'anomaly_dataset'  # Your dataset root path
result = 'anomaly_frame_dataset'  # Output path for frames
video_extensions = ['.mp4', '.avi', '.mov', '.mkv']  # Video file types to consider

# ✅ Loop over Parts (e.g., Anomaly-Videos-Part-1 to Part-4)
for part in tqdm(os.listdir(path), desc="Processing Dataset Parts"):
    part_path = os.path.join(path, part)
    result_part_path = os.path.join(result, part)
    os.makedirs(result_part_path, exist_ok=True)

    # ✅ Loop over categories (e.g., Abuse, Arrest, Explosion)
    for category in tqdm(os.listdir(part_path), desc=f"Processing {part}", leave=False):
        category_path = os.path.join(part_path, category)
        result_category_path = os.path.join(result_part_path, category)
        os.makedirs(result_category_path, exist_ok=True)

        # ✅ Loop over video files inside each category
        for video_file in os.listdir(category_path):
            vid_path = os.path.join(category_path, video_file)
            result_video_path = os.path.join(result_category_path, video_file[:-4])  # Output folder per video
            os.makedirs(result_video_path, exist_ok=True)

            # ✅ Check if it's a video file (skip directories or other file types)
            if not os.path.isfile(vid_path) or not any(video_file.lower().endswith(ext) for ext in video_extensions):
                print(f"⚠️ Skipping non-video file or directory: {vid_path}")
                continue

            # ✅ Extract and save frames
            frames = extract_uniform_frames(vid_path, num_frames=16)
            if frames:  # Save if frames were successfully extracted
                for idx, frame in enumerate(frames):
                    frame.save(os.path.join(result_video_path, f"{idx}.jpg"))

Processing Dataset Parts:   0%|          | 0/4 [00:00<?, ?it/s]

Processing Anomaly-Videos-Part-1:   0%|          | 0/4 [00:00<?, ?it/s]

Processing Anomaly-Videos-Part-2:   0%|          | 0/2 [00:00<?, ?it/s]

Processing Anomaly-Videos-Part-3:   0%|          | 0/2 [00:00<?, ?it/s]

Processing Anomaly-Videos-Part-4:   0%|          | 0/2 [00:00<?, ?it/s]

# Process normal videos

In [19]:
import zipfile
import os

# List of zip files
zip_file = 'Normal_Videos_for_Event_Recognition.zip'


# Output directory to extract files
output_dir = 'normal_dataset'

# Make sure output directory exists
os.makedirs(output_dir, exist_ok=True)


with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    print(f"Extracting {zip_file}...")
    zip_ref.extractall(output_dir)

print("All files extracted successfully!")


Extracting Normal_Videos_for_Event_Recognition.zip...
All files extracted successfully!


In [20]:
# ✅ Paths
input_path = 'normal_dataset/Normal_Videos_for_Event_Recognition'  # Input path containing all normal videos
output_path = 'normal_frame_dataset'  # Output path to save extracted frames
os.makedirs(output_path, exist_ok=True)

video_extensions = ['.mp4', '.avi', '.mov', '.mkv']  # Supported video formats

# ✅ Process each video file
for video_file in tqdm(os.listdir(input_path), desc="Processing Normal Videos"):
    vid_path = os.path.join(input_path, video_file)

    # ✅ Skip non-video files
    if not os.path.isfile(vid_path) or not any(video_file.lower().endswith(ext) for ext in video_extensions):
        print(f"⚠️ Skipping non-video file: {vid_path}")
        continue

    # ✅ Create output folder for each video
    result_video_path = os.path.join(output_path, video_file[:-4])  # Remove .mp4 extension
    os.makedirs(result_video_path, exist_ok=True)

    # ✅ Extract and save frames
    frames = extract_uniform_frames(vid_path, num_frames=16)
    if frames:
        for idx, frame in enumerate(frames):
            frame.save(os.path.join(result_video_path, f"{idx}.jpg"))


Processing Normal Videos:   0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
# path = '/content/Dataset'
# res = '/content/crime16'
# #Number
# seq_length = 16

# def preprocess_data(seq_length,path,res):
#   dir = os.listdir(path)
#   for i in tqdm(dir):
#       p1 = os.path.join(path,i)
#       r1 = os.path.join(res,i)
#       os.makedirs(r1,exist_ok = True)
#       for j in os.listdir(p1):
#           p2 = os.path.join(p1,j)
#           r2 = os.path.join(r1,j)
#           l = 0
#           skip_length = int(len(os.listdir(p2))/seq_length)
#           for m in range(10):
#               k = m
#               while(l!=seq_length):

#                   p3 = os.path.join(p2,str(k) + ".jpg")
#                   try:
#                       img = cv2.imread(p3)
#                       img = cv2.resize(img,(128,128))
#                   except:
#                       print(p3)
#                   if(k==0):
#                       img1 = img
#                   else:
#                       img1 = np.append(img1,img,axis = 1)
#                   k = k+skip_length
#                   l = l+1
#               cv2.imwrite(r2 + str(m)+".jpg",img1)


In [27]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset
from torchvision import transforms
import numpy as np

class VideoFrameDataset(Dataset):
    def __init__(self, root_dir, seq_length=16, transform=None):
        self.root_dir = root_dir
        self.seq_length = seq_length
        self.transform = transform

        # ✅ Only collect folders that contain frame files
        self.video_folders = []
        for root, dirs, files in os.walk(root_dir):
            if len(files) > 0:  # This folder has files (frames)
                self.video_folders.append(root)

        print(f"✅ Found {len(self.video_folders)} video frame folders.")

    def __len__(self):
        return len(self.video_folders)

    def __getitem__(self, idx):
        video_folder = self.video_folders[idx]
        frame_files = sorted(os.listdir(video_folder))
        total_frames = len(frame_files)

        # ✅ If empty folder (shouldn't happen but for safety)
        if total_frames == 0:
            raise RuntimeError(f"⚠️ Empty folder found: {video_folder}")

        # ✅ Sample frame indices
        if total_frames < self.seq_length:
            indices = list(range(total_frames)) + [total_frames - 1] * (self.seq_length - total_frames)
        else:
            indices = np.linspace(0, total_frames - 1, self.seq_length, dtype=int)

        # ✅ Load and transform frames
        frames = []
        for i in indices:
            frame_path = os.path.join(video_folder, frame_files[i])
            image = Image.open(frame_path).convert('RGB')
            if self.transform:
                image = self.transform(image)
            frames.append(image)

        # Stack to [C, T, H, W]
        frames = torch.stack(frames, dim=0).permute(1, 0, 2, 3)  # [T, C, H, W] -> [C, T, H, W]

        # ✅ Example label: 0 for Normal, 1 for Anomaly (based on path)
        label = self.get_label(video_folder)

        return frames, label

    def get_label(self, video_folder):
        # Example rule: You can adjust this depending on your real folder names
        if "Normal" in video_folder:
            return torch.tensor(0)  # Normal class
        else:
            return torch.tensor(1)  # Anomaly class


In [28]:
from torch.utils.data import DataLoader
from torchvision import transforms

# Image transformations
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Resize frames
    transforms.ToTensor(),  # Convert to tensor
])

# Dataset and DataLoader
dataset = VideoFrameDataset('anomaly_frame_dataset', seq_length=16, transform=transform)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True, num_workers=0)


✅ Found 650 video frame folders.


In [29]:
from ultralytics.utils.torch_utils import select_device

device = select_device("")
print("Using device:", device)

Ultralytics 8.3.38  Python-3.11.9 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3050 4GB Laptop GPU, 4096MiB)
Using device: cuda:0


In [42]:
# Final Balanced Training Script with Augmentation, Class Weights, and Sampler

import os
import torch
import numpy as np
from PIL import Image
from torch.utils.data import Dataset, DataLoader, random_split, WeightedRandomSampler
from torchvision import transforms
import torch.nn as nn
import torch.optim as optim
from model import resnet50  # Import your SlowFast model
from clr import OneCycle, update_lr, update_mom  # Scheduler

# ------------------------- CONFIG ------------------------------
SEQ_LENGTH = 16
FRAME_SIZE = (128, 128)
BATCH_SIZE = 8
EPOCHS = 20
LEARNING_RATE = 0.0005  # Reduced learning rate
ANOMALY_PATH = 'anomaly_frame_dataset'
NORMAL_PATH = 'normal_frame_dataset'
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# ---------------------------------------------------------------


# ------------------- Dataset Definition -----------------------
class VideoFrameDataset(Dataset):
    def __init__(self, root_dir, label, seq_length=16, transform=None):
        self.root_dir = root_dir
        self.seq_length = seq_length
        self.label = label  # 0 for normal, 1 for anomaly
        self.transform = transform

        # Collect all folders containing frames
        self.video_folders = []
        for root, dirs, files in os.walk(root_dir):
            if len(files) > 0:
                self.video_folders.append(root)

        print(f"Found {len(self.video_folders)} video frame folders in {root_dir}.")

    def __len__(self):
        return len(self.video_folders)

    def __getitem__(self, idx):
        video_folder = self.video_folders[idx]
        frame_files = sorted(os.listdir(video_folder))
        total_frames = len(frame_files)

        # Ensure at least seq_length frames
        if total_frames < self.seq_length:
            indices = list(range(total_frames)) + [total_frames - 1] * (self.seq_length - total_frames)
        else:
            indices = np.linspace(0, total_frames - 1, self.seq_length, dtype=int)

        frames = []
        for i in indices:
            frame_path = os.path.join(video_folder, frame_files[i])
            image = Image.open(frame_path).convert('RGB')
            if self.transform:
                image = self.transform(image)
            frames.append(image)

        # Stack and permute
        frames = torch.stack(frames, dim=0).permute(1, 0, 2, 3)  # [C, T, H, W]

        return frames, torch.tensor(self.label)


# ------------------- Data Preparation -------------------------
# Transformations including augmentation and normalization
transform = transforms.Compose([
    transforms.Resize(FRAME_SIZE),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Load datasets
anomaly_dataset = VideoFrameDataset(ANOMALY_PATH, label=1, seq_length=SEQ_LENGTH, transform=transform)
normal_dataset = VideoFrameDataset(NORMAL_PATH, label=0, seq_length=SEQ_LENGTH, transform=transform)

# Combine datasets
full_dataset = torch.utils.data.ConcatDataset([anomaly_dataset, normal_dataset])

# Class counts
anomaly_count = len(anomaly_dataset)
normal_count = len(normal_dataset)
total = anomaly_count + normal_count

# Class weights (inverse frequency for balance)
w_normal = total / (2 * normal_count)
w_anomaly = total / (2 * anomaly_count)
weights = torch.tensor([w_normal, w_anomaly]).to(DEVICE)
print(f"Adjusted Class weights -> Normal: {w_normal:.2f}, Anomaly: {w_anomaly:.2f}")

# Sample weights for balancing
sample_weights = [w_anomaly] * anomaly_count + [w_normal] * normal_count
sampler = WeightedRandomSampler(sample_weights, num_samples=len(sample_weights), replacement=True)

# ------------------- Train/Validation Split -------------------
val_split = 0.2
num_val = int(len(full_dataset) * val_split)
num_train = len(full_dataset) - num_val

train_dataset, val_dataset = random_split(full_dataset, [num_train, num_val])

# Get lengths of anomaly and normal in training dataset (estimate proportionally)
num_anomaly = int(anomaly_count * (num_train / total))
num_normal = int(normal_count * (num_train / total))

# Recompute weights based on split
sample_weights = [w_anomaly] * num_anomaly + [w_normal] * num_normal
sampler = WeightedRandomSampler(sample_weights, num_samples=len(sample_weights), replacement=True)

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, sampler=sampler, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)



# ------------------- Model & Training Setup -------------------
model = resnet50(class_num=2).to(DEVICE)
criterion = nn.CrossEntropyLoss(weight=weights)
optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=0.9, weight_decay=1e-4)
scheduler = OneCycle(nb=len(train_loader) * EPOCHS, max_lr=LEARNING_RATE)


# ------------------- Training Loop -------------------
print("Starting Training...")
for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)

        # Forward
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Scheduler
        lr, mom = scheduler.calc()
        update_lr(optimizer, lr)
        update_mom(optimizer, mom)

        running_loss += loss.item()

    # Validation
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

    print(f"Epoch [{epoch+1}/{EPOCHS}] | Train Loss: {running_loss/len(train_loader):.4f} | Val Loss: {val_loss/len(val_loader):.4f}")

print("Training completed!")
torch.save(model.state_dict(), 'slowfast_crime_detection_balanced.pth')
print("Model saved as 'slowfast_crime_detection_balanced.pth'.")

Found 650 video frame folders in anomaly_frame_dataset.
Found 50 video frame folders in normal_frame_dataset.
Adjusted Class weights -> Normal: 7.00, Anomaly: 0.54
Starting Training...
Epoch [1/20] | Train Loss: 1.0614 | Val Loss: 1.9394
Epoch [2/20] | Train Loss: 1.0631 | Val Loss: 2.3492
Epoch [3/20] | Train Loss: 1.3757 | Val Loss: 0.4761
Epoch [4/20] | Train Loss: 0.9670 | Val Loss: 1.1783
Epoch [5/20] | Train Loss: 1.0474 | Val Loss: 0.3125
Epoch [6/20] | Train Loss: 1.2391 | Val Loss: 1.9959
Epoch [7/20] | Train Loss: 1.1080 | Val Loss: 2.7139
Epoch [8/20] | Train Loss: 1.3821 | Val Loss: 1.8716
Epoch [9/20] | Train Loss: 1.4422 | Val Loss: 0.5136
Epoch [10/20] | Train Loss: 1.2802 | Val Loss: 2.1546
Epoch [11/20] | Train Loss: 1.1316 | Val Loss: 0.4588
Epoch [12/20] | Train Loss: 1.6406 | Val Loss: 0.6856
Epoch [13/20] | Train Loss: 0.9690 | Val Loss: 0.4502
Epoch [14/20] | Train Loss: 1.0249 | Val Loss: 0.3363
Epoch [15/20] | Train Loss: 0.8087 | Val Loss: 0.2778
Epoch [16/20] 

In [31]:
# ✅ Save model weights after training
torch.save(model.state_dict(), 'slowfast_crime_detection.pth')
print("✅ Model weights saved to 'slowfast_crime_detection.pth'")

✅ Model weights saved to 'slowfast_crime_detection.pth'
